# langChain Quickstart

## Initialization
### Environment and APIs

In [1]:
from dotenv import load_dotenv
load_dotenv()

# import os
# print ('LANGCHAIN_API_KEY: ', os.getenv('LANGCHAIN_API_KEY'))
# print ('OPENAI_API_KEY: ', os.getenv('OPENAI_API_KEY'))
# print ('TAVILY_API_KEY: ', os.getenv('TAVILY_API_KEY'))

True

### Models

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatOpenAI()
output_parser = StrOutputParser()


## Docstore pre-load

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()

docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


## Chains - Document Retrieval

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

doc_prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(chat_model, doc_prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

_TEST - retrieval_chain_

In [ ]:
retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

## Chains - History Aware Retrieval
### Generate search from history

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

search_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to lookup in order to get information relevant to the conversation.")
])
history_retriever_chain = create_history_aware_retriever(chat_model, retriever, search_prompt)


### Generate answer from history

In [6]:
history_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(chat_model, history_prompt)
history_chain = create_retrieval_chain(history_retriever_chain, document_chain)


_TEST - history_chain_

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

## Agent
### Search tool

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [8]:
tools = [retriever_tool, search]

In [10]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

agent_prompt = hub.pull("hwchase17/openai-functions-agent")
agent_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(agent_chat_model, agent_prompt, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


ImportError: Could not import langchainhub, please install with `pip install langchainhub`.